

# STEP1: 전처리

## 1-0: 전체 파일을 코인 별로 나누어 저장

<!-- 전체파일을 sort_values 해보자. (시간순으로 정렬) -->

In [1]:
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

# Change "current directory"
new_dir = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
# new_dir = 'C:\\Users\\user\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(new_dir)

import numpy as np
import csv
import dask.dataframe as dd # This is a main package to process a large csv file.
import pyarrow as pa
import pyarrow.parquet as pq
import pickle
import pandas as pd 


In [2]:
# data_id = 202302230905 
# data_id = 202311281936 
# data_id = 202303271051
# data_id = 202312021632 
# data_id = 202312081015
# data_id = '202312122122'
# 202302230905
#  202303271051 # 실질적으로는 12/16부터 2/26까지 데이터 존재.

# in_csv ="D:\\ticker_data_{}.csv".format(data_id)
in_csv ="D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\Data\\ticker_data_{}.csv".format(data_id)

dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# Read the CSV file using a Dask DataFrame
df = dd.read_csv(in_csv, dtype=dtype) 


NameError: name 'data_id' is not defined

In [3]:
## 수집된 파일에는 n개의 코인이 함께 저장되어 있음. 따라서, Separate files by COIN

coin_list = ['BTC'] #, 'ETH', 'DOGE', 'XRP', 'LINK', 'AVAX', 'SOL']

for coin in coin_list:
    globals()["df_temp"] = globals()["df"][globals()["df"].code == f'KRW-{coin}']
    # with open("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv".format(coin, data_id), 'wb') as file:
    #     pickle.dump(df_temp, file)
    df_temp.to_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv".format(coin, data_id))
    print('Complete: {}'.format(coin))


KeyError: 'df'

## 1-1: 코인별로 나누어진 파일을, 시간순으로 sorting 후 다시 저장
### duplicates 제거 및 30초/60초 ticker 값을 제거


In [ ]:
import os
#USE ONLY ONE OF THESE:
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

# Change "current directory"
new_dir = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(new_dir)

# Import packages
import numpy as np
import csv
import dask.dataframe as dd # This is a main package to process a large csv file.
import pyarrow as pa
import pyarrow.parquet as pq
# import pandas as pd 

In [ ]:
dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# data_id = 202303271051 # 해당 틱데이터 커버 기간: 22년 12월 16일 오후 9시 ~ 23년 2월 26일 오전 4시
# coin_list = [ 'BTC',  'ETH',  'DOGE', 'XRP'] 

for coin in coin_list:

    # Read dask "part" files
    part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv\\".format(coin, data_id)
    df = dd.read_csv( part_files_path + '*.part', dtype=dtype)    
    # df = dd.read_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\t_data_{}_{}.csv".format(coin, data_id))
    # with open('.\\output\\t_data_{}_{}.pkl'.format(coin, data_id), 'rb') as file:
    #     loaded_df = pickle.load(file)

    # Sort dataframe
    df = df.sort_values('sys_datetime', ascending = True, na_position = 'last')
    print('Complete: Sorting')
    
    # Write the sorted DataFrame to a new dask-type csv file
    df.to_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_data_{}_{}.csv".format(coin, data_id), index=False)

    # with open("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_data_{}_{}.csv".format(coin, data_id), 'wb') as file:
    #     pickle.dump(loaded_df, file)

    # # Write the sorted DataFrame to a new parquet file
    # df.to_parquet('s_t_data_{}_{}.parquet'.format(coin,data_id), engine='pyarrow')

## Note: 용량 부족 메시지가 떠서, 한 코인씩 돌림. (for 문을 돌리지 못함.)    
## Note: parquet으로는 저장되지 않음 (메모리 문제라는 메시지가 뜸.)

Complete: Sorting


### duplicates 제거 및 30초/60초 ticker 값을 제거 (단위 시간마다 신규 거래가 없더라도 ticker 값이 들어온다)

In [18]:
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
new_dir = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(new_dir)
import numpy as np
import csv
import dask.dataframe as dd
import pyarrow as pa
import pyarrow.parquet as pq
import gc


dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

data_id = '202312122122'
# data_id = '202303271051' # 해당 틱데이터 커버 기간: 22년 12월 16일 오후 9시 ~ 23년 2월 26일 오전 4시
# coin_list = ['BTC', 'ETH',  'DOGE', 'XRP'] 
coin_list = ['BTC'] 
for coin in coin_list:

    # with open('.\\output\\s_t_data_{}_{}.csv'.format(coin, data_id), 'rb') as file:
    #     loaded_df = pickle.load(file)

    ### Read dask "part" failes
    part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_data_{}_{}.csv\\".format(coin, data_id)
    df = dd.read_csv( part_files_path + '*.part', dtype=dtype)    
    df = df.compute()

    gc.collect()

    ### Drop duplicates
    print("df.shape 1 :", df.shape)
    df.drop_duplicates(subset=[ 'type_websocket', 'timestamp','sys_datetime'], keep='first', inplace=True, ignore_index=True)
    print("df.shape 2 :", df.shape)
    df.drop_duplicates(subset=[ 'type_websocket',  'opening_price',  'high_price',   'low_price',  'trade_price',  'prev_closing_price', 'change',  'change_price',  'signed_change_price',  'change_rate',  'signed_change_rate',  'trade_volume',  'acc_trade_volume',   'acc_trade_price',    'trade_date', 'ask_bid',  'acc_ask_volume',  'acc_bid_volume',  'highest_52_week_price', 'highest_52_week_date',  'lowest_52_week_price', 'lowest_52_week_date', 'market_state',  'is_trading_suspended'   , 'total_ask_size',  'total_bid_size',  'orderbook_ap_0',  'orderbook_as_0',  'orderbook_bp_0',  'orderbook_bs_0',  'orderbook_ap_1',  'orderbook_as_1',  'orderbook_bp_1',  'orderbook_bs_1',  'orderbook_ap_2',  'orderbook_as_2',  'orderbook_bp_2',  'orderbook_bs_2',  'orderbook_ap_3',  'orderbook_as_3',  'orderbook_bp_3',  'orderbook_bs_3',  'orderbook_ap_4',  'orderbook_as_4',  'orderbook_bp_4',  'orderbook_bs_4',  'orderbook_ap_5',  'orderbook_as_5',  'orderbook_bp_5',  'orderbook_bs_5',  'orderbook_ap_6',  'orderbook_as_6',  'orderbook_bp_6',  'orderbook_bs_6',  'orderbook_ap_7',  'orderbook_as_7',  'orderbook_bp_7',  'orderbook_bs_7',  'orderbook_ap_8',  'orderbook_as_8',  'orderbook_bp_8',  'orderbook_bs_8',  'orderbook_ap_9',  'orderbook_as_9',  'orderbook_bp_9',  'orderbook_bs_9',  'orderbook_ap_10',  'orderbook_as_10',  'orderbook_bp_10',  'orderbook_bs_10',  'orderbook_ap_11',  'orderbook_as_11',  'orderbook_bp_11',  'orderbook_bs_11',  'orderbook_ap_12',  'orderbook_as_12',  'orderbook_bp_12',  'orderbook_bs_12',  'orderbook_ap_13',  'orderbook_as_13',  'orderbook_bp_13',  'orderbook_bs_13',  'orderbook_ap_14',  'orderbook_as_14',  'orderbook_bp_14',  'orderbook_bs_14'    ], keep='first', inplace=True, ignore_index=True)
    print("df.shape 3 :", df.shape)

    print('Complete: Drop duplicates')

    ### Write the sorted DataFrame to a new dask-type csv file
    df.to_csv("D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_d_data_{}_{}.csv".format(coin, data_id), index=False)
    print('Complete: File Saving')


df.shape 1 : (10606368, 99)
df.shape 2 : (10606368, 99)
df.shape 3 : (9506748, 99)
Complete: Drop duplicates
Complete: File Saving


## 1-2(b): 각 코인의 틱데이터를 10분씩 묶는 전처리 작업 (rolling window)

### Setting & Running => Output: {}_sum_both_10m.csv

In [1]:
# Version Number

data_id = 202303271051
# data_id = 202312122122
coin = 'XRP'

### Libraries and key functions
import os
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(working_directory)

import numpy as np
import csv
import gc
import dask.dataframe as dd
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from contextlib import contextmanager  # 컨텍스트 관리자를 사용하기 위한 contextlib 모듈을 가져옵니다.
import time  # 시간 관련 기능을 사용하기 위한 time 모듈을 가져옵니다.


def log_return(series: np.ndarray):
    return np.log(series).diff()

def realized_volatility(series):
    return np.sqrt(np.sum(series ** 2))

def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('.'.join([prefix] + list(c)))
    return ret

@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f'[{name}] {elapsed: .3f}초')


In [2]:
import os
#USE ONLY ONE OF THESE:
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask
# Change "current directory"
working_directory = 'D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\'
os.chdir(working_directory)

import numpy as np
import csv
import dask.dataframe as dd # This is a main package to process a large csv file.
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd 
import gc

dtype={'ask_bid': 'object',
       'change': 'object',
       'highest_52_week_date': 'object',
       'lowest_52_week_date': 'object',
       'market_state': 'object',
       'market_warning': 'object',
       'orderbook_ap_0': 'float32',
       'orderbook_ap_1': 'float32',
       'orderbook_ap_2': 'float32',
       'orderbook_ap_3': 'float32',
       'orderbook_bp_0': 'float32',
       'orderbook_bp_1': 'float32',
       'orderbook_bp_10': 'float32',
       'orderbook_bp_11': 'float32',
       'orderbook_bp_12': 'float32',
       'orderbook_bp_13': 'float32',
       'orderbook_bp_14': 'float32',
       'orderbook_bp_2': 'float32',
       'orderbook_bp_3': 'float32',
       'orderbook_bp_4': 'float32',
       'orderbook_bp_5': 'float32',
       'orderbook_bp_6': 'float32',
       'orderbook_bp_7': 'float32',
       'orderbook_bp_8': 'float32',
       'orderbook_bp_9': 'float32',
       'stream_type': 'object',
       'trade_time': 'object'}

# with open('.\\output\\s_t_d_data_{}_{}.pkl'.format(coin, data_id), 'rb') as file:
#        df = pickle.load(file)
# s_t_d_data_BTC_202312122122

## 저장이 Dask dataframe으로 되어 있으면 아래 코드를 실행
part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_d_data_{}_{}.csv\\".format(coin, data_id)
df = dd.read_csv( part_files_path + '*.part', dtype=dtype) 

# # ## 저장이 pandas dataframe으로 되어 있으면 아래 코드를 실행
# part_files_path =  "D:\\OneDrive - 한동대학교\\PROJECT\\트머프로젝트\\output\\s_t_d_data_{}_{}.csv".format(coin, data_id)
# df = pd.read_csv( part_files_path , dtype=dtype) 


### Feature Engineering (TICKER 부문)

In [3]:
def feature_engineering_ticker(df_ticker):

    df_ticker_pd = df_ticker.copy()
    df_ticker_pd.reset_index(inplace=True, drop=True)
    df_ticker_pd['datetime'] = pd.to_datetime(df_ticker_pd['datetime'])
    # df_ticker_pd['log_return'] = df_ticker_pd['trade_price'].pct_change().apply(lambda x: np.log(1 + x))

    # # signed volume (to calculate volume power)
    # df_ticker_pd['ask_bid'] = df_ticker_pd['ask_bid'].map({'ASK': -1, 'BID': 1})
    # df_ticker_pd['signed_volume'] = df_ticker_pd['trade_volume'] * df_ticker_pd['ask_bid'] 

    # Set 'timestamp' as the index for resampling
    df_ticker_pd.set_index('datetime', inplace=True)
    # Resample data into 10-minute bins and take the last value in each bin
    df_ticker_pd_1m = df_ticker_pd['trade_price'].resample('1T').last()
    # Shift the 1-minute prices one step forward to align with the original seconds data
    df_ticker_pd_1m = df_ticker_pd_1m.shift(1)
    # Now, reindex the original DataFrame with 10-minute frequency and forward fill the 'ending_price' values
    df_ticker_pd['ending_price_b1m'] = df_ticker_pd_1m.reindex(df_ticker_pd.index, method='ffill') # 이를 이용해서 10분 동안 얼마나 하락하는가 확인.
    # Reset the index to get the DataFrame with the original 'datetime', 'trade_price', and 'ending_price' columns
    df_ticker_pd.reset_index(inplace=True)
    # datetime을 1분 단위로 정리하는 변수 (dt_1m)을 만듬 (이는 데이터를 '정리'하는 데에 활용될 단위임)
    df_ticker_pd['dt_1m'] = df_ticker_pd['datetime'].dt.floor('1min').dt.strftime('%Y-%m-%d %H:%M:%S')

    # Set 'dt_1m' as the index for easier resampling
    df_ticker_pd['dt_1m'] = pd.to_datetime(df_ticker_pd['dt_1m'])
    df_ticker_pd.set_index('dt_1m', inplace=True, drop = False)
    # Define the size of the moving window (10 minutes in this case)
    window_size = pd.Timedelta(minutes=10)

    # Create an empty list to store features for each rolling window
    # realized_volatility = []
    realized_volatility_mean0 = []
    realized_up_volatility_mean0 = []
    realized_down_volatility_mean0 = []
    num_trades = []
    lowest_return = []
    trade_vol = []
    volume_power = []
    highest_return = []
    high_low_gap = []
    end_price = []
    # prices_30s_for_NN = []
    # num_of_trades = []
    returns = []
    trade_vol_top1p_ratio = []
    volume_power_top1p = []

    temp_index = df_ticker_pd.index.unique().tolist()

    gc.collect()
    
    # Define the time range
    start_time = pd.to_datetime('00:00:00').time()
    end_time = pd.to_datetime('06:00:00').time()

    window_start2 = []
    window_end2 = []

    # Iterate through each time window using the rolling method
    for window_start in temp_index[:-10]:
        window_end = window_start + window_size

        if  (start_time <= window_start.time() <= end_time) or  (start_time <= window_end.time() <= end_time):
            continue 

        

        # print("window_start:", window_start, " & window_end:", window_end)

        # Filter the data for each rolling window
        rolling_window_data = df_ticker_pd.loc[(df_ticker_pd.index >= window_start) & (df_ticker_pd.index < window_end)]
        # previous_price = rolling_window_data['ending_price_b1m'].iloc[0] # 10분 bin이 시작하기 직전 마지막 trade price 값을 의미.

        try: 
            previous_price = rolling_window_data['trade_price'].iloc[0]
        except Exception as e:
            print("No previous price.")
            continue

        window_start2.append(window_start)
        window_end2.append(window_end)
        
        # rolling_window_data['log_return_from_pp'] = np.log(rolling_window_data['trade_price'] / previous_price)
        rolling_window_data.loc[:, 'log_return_from_pp'] = np.log(rolling_window_data['trade_price'] / previous_price)

        # VARIABLE: realized_volatility 
        # Calculate the returns (percentage change) for each tick in the rolling window
        returns2 = rolling_window_data['log_return_from_pp']
        # Calculate the volatility as the standard deviation of returns
        # volatility = returns.std()
        # Append the average volatility to the list
        # realized_volatility.append(volatility)

        # returns_from_pp = rolling_window_data['log_return_from_pp']
        squared_returns = np.square(returns2)
        average_squared_returns = np.mean(squared_returns)
        realized_volatility_mean0.append(np.sqrt(average_squared_returns))

        squared_returns_up = np.square(returns2[returns2 >= 0])
        squared_returns_down = np.square(returns2[returns2 < 0])
        average_squared_returns_up = np.mean(squared_returns_up)
        average_squared_returns_down = np.mean(squared_returns_down)
        realized_up_volatility_mean0.append(np.sqrt(average_squared_returns_up))
        realized_down_volatility_mean0.append(np.sqrt(average_squared_returns_down))

        # VARIABLE: number of trades 
        # Calculate the volatility as the standard deviation of returns
        n_trades = rolling_window_data.shape[0]
        # Append n_trades to the list
        num_trades.append(n_trades)

        # VARIABLE: lowest return
        # Calculate the lowest return
        l_return = rolling_window_data['log_return_from_pp'].min()
        # Append n_trades to the list
        lowest_return.append(l_return)

        # VARIABLE: highest return
        h_return = rolling_window_data['log_return_from_pp'].max()
        highest_return.append(h_return) 

        # VARIABLE: high_low_gap
        hl_gap = h_return - l_return
        high_low_gap.append(hl_gap)

        # VARIABLE: total trade volume
        # Calculate the trading volume during the 10m
        tv = rolling_window_data['trade_volume'].sum()
        # Append n_trades to the list
        trade_vol.append(tv)

        # Assuming df is your DataFrame and 'A' is the column of interest
        percentile_99 = rolling_window_data['trade_volume'].quantile(0.99)
        top1p = rolling_window_data[rolling_window_data['trade_volume'] >= percentile_99]
        tv_top1p = rolling_window_data[rolling_window_data['trade_volume'] >= percentile_99]['trade_volume'].sum()
        tv_top1p_rate = tv_top1p/tv
        trade_vol_top1p_ratio.append(tv_top1p_rate)

        # VARIABLE: volume power
        # Calculate the trading volume during the 10m
        ask_sum = rolling_window_data[rolling_window_data['ask_bid'] == 'ASK']['trade_volume'].sum()
        bid_sum = rolling_window_data[rolling_window_data['ask_bid'] == 'BID']['trade_volume'].sum()
        vp = (bid_sum - ask_sum) / tv  # 후에 전체 거래량으로 나누어줄 예정이다.
        volume_power.append(vp)

        ask_sum_top1p = top1p[top1p['ask_bid'] == 'ASK']['trade_volume'].sum()
        bid_sum_top1p = top1p[top1p['ask_bid'] == 'BID']['trade_volume'].sum()
        vp_top1p = (bid_sum_top1p - ask_sum_top1p) / tv_top1p # 후에 전체 거래량으로 나누어줄 예정이다.
        volume_power_top1p.append(vp_top1p)

        ep = rolling_window_data['trade_price'].iloc[-1]
        end_price.append(ep)

        rts = rolling_window_data['trade_price'].iloc[-1]/previous_price
        returns.append(rts)

        # VARIABLE: prices_30s_for_NN
        # rolling_window_data.set_index('datetime', inplace=True, drop = False)
        # temp = rolling_window_data['log_return_from_pp'].resample('30S').last()
        # prices_30s_for_NN.append(temp)

    # Create a new DataFrame to store the results

    # print("len(window_end):",len(temp_index[10:]))
    # print("len(window_start):",len(temp_index[:-10]))
    # print("len(realized_volatility_mean0):",len(realized_volatility_mean0))
    # print("len(realized_up_volatility_mean0):",len(realized_up_volatility_mean0))
    # print("len(realized_down_volatility_mean0):",len(realized_down_volatility_mean0))
    # print("len(num_trades):",len(num_trades))
    # print("len(lowest_return):",len(lowest_return))
    # print("len(highest_return):",len(highest_return))
    # print("len(high_low_gap):",len(high_low_gap))
    # print("len(trade_vol):",len(trade_vol))
    # print("len(trade_vol_top1p_ratio):",len(trade_vol_top1p_ratio))
    # print("len(volume_power):",len(volume_power))
    # print("len(volume_power_top1p):",len(volume_power_top1p))
    # print("len(end_price):",len(end_price))
    # print("len(returns):",len(returns))

    result_df = pd.DataFrame({
        'window_start': window_start2,
        'window_end': window_end2,
        # 'realized_volatility': realized_volatility,
        'realized_volatility_mean0':realized_volatility_mean0,
        'realized_up_volatility_mean0':realized_up_volatility_mean0,
        'realized_down_volatility_mean0':realized_down_volatility_mean0,
        'num_trades': num_trades,
        'lowest_return': lowest_return,
        'highest_return': highest_return,
        'high_low_gap': high_low_gap,
        'trade_vol': trade_vol,
        'trade_vol_top1p_ratio': trade_vol_top1p_ratio,
        'volume_power': volume_power,
        'volume_power_top1p': volume_power_top1p,
        'end_price': end_price,
        'returns': returns,
        # 'prices_30s_for_NN': prices_30s_for_NN,
    })

    # Reset the index of the result DataFrame
    result_df.reset_index(drop=True, inplace=True)

    # 10분동안의 return을 안 만들었었네//
    # result_df['return'] = np.log(result_df['end_price'].shift(-10) / result_df['end_price'])

    result_df['return'] = result_df['returns']
    result_df = result_df.drop('returns', axis=1)

    # time_id 만들어주기.
    result_df['window_start'] = pd.to_datetime(result_df['window_start'])
    result_df['time_id'] = result_df['window_start'].dt.floor('1min').dt.strftime('%Y-%m-%d %H:%M:%S')
    result_df['time_id'] = pd.to_datetime(result_df['time_id'])

    # Bolinger Band 
    result_df['log_return_temp'] = result_df['end_price'].pct_change().apply(lambda x: np.log(1 + x))

    result_df['BB_width_w20'] = result_df['log_return_temp'].rolling(20).std()*4
    result_df['BB_width_w40'] = result_df['log_return_temp'].rolling(40).std()*4
    result_df['BB_width_w10'] = result_df['log_return_temp'].rolling(10).std()*4

    # 종속변수 만들기 (향후 10분의 realized vol / lowest return)
    df_ticker_pd_1m = df_ticker_pd_1m.shift(1)
    # result_df['dv1_realized_volatility'] = result_df['realized_volatility'].shift(-10)
    result_df['dv5_realized_volatility_mean0'] = result_df['realized_volatility_mean0'].shift(-10)
    result_df['dv2_lowest_return'] = result_df['lowest_return'].shift(-10)
    result_df['dv3_highest_return'] = result_df['highest_return'].shift(-10)

    ##### 450/300/150초 기준 변수 만들기 => 직전 1분. 직전 5분 으로 수정.

    # Iterate through each time window using the rolling method

    time_intervals = [60, 300]

    for interval in time_intervals:

        # realized_volatility = []
        realized_volatility_mean0 = []
        num_trades = []
        lowest_return = []
        highest_return = []
        trade_vol = []
        high_low_gap = []
        volume_power = []
        trade_vol_top1p_ratio = []
        volume_power_top1p = []
        window_start2 = []
        window_end2 = []

        print("###interval:", interval)
        for window_start in temp_index[:-10]:
            window_end = window_start + window_size

            if  (start_time <= window_start.time() <= end_time) or  (start_time <= window_end.time() <= end_time):
                continue 
            
            
            # Filter the data for each rolling window
            rolling_window_data = df_ticker_pd.loc[(df_ticker_pd['datetime'] >= window_end - pd.Timedelta(seconds=interval)) & (df_ticker_pd['datetime'] < window_end)]
            # previous_price = rolling_window_data['trade_price'].iloc[0] # 10분 bin이 시작하기 직전 마지막 trade price 값을 의미.
            
            try: 
                previous_price = rolling_window_data['trade_price'].iloc[0]
            except Exception as e:
                print("No previous price.")
                continue

            window_start2.append(window_start)
            window_end2.append(window_end)

            rolling_window_data.loc[:, 'log_return_from_pp'] = np.log(rolling_window_data['trade_price'] / previous_price)

            # VARIABLE: realized_volatility 
            returns = rolling_window_data['log_return_from_pp']
            # volatility = returns.std()
            # realized_volatility.append(volatility)

            squared_returns = np.square(returns)
            average_squared_returns = np.mean(squared_returns)
            realized_volatility_mean0.append(np.sqrt(average_squared_returns))

            # VARIABLE: number of trades 
            n_trades = rolling_window_data.shape[0]
            num_trades.append(n_trades)

            # VARIABLE: lowest return
            l_return = rolling_window_data['log_return_from_pp'].min()
            lowest_return.append(l_return)

            # VARIABLE: highest return
            h_return = rolling_window_data['log_return_from_pp'].max()
            highest_return.append(h_return) 

            # VARIABLE: high_low_gap
            hl_gap = h_return - l_return
            high_low_gap.append(hl_gap)

            # VARIABLE: total trade volume
            # Calculate the trading volume during the 10m
            tv = rolling_window_data['trade_volume'].sum()
            # Append n_trades to the list
            trade_vol.append(tv)

            # Assuming df is your DataFrame and 'A' is the column of interest
            percentile_99 = rolling_window_data['trade_volume'].quantile(0.99)
            top1p = rolling_window_data[rolling_window_data['trade_volume'] >= percentile_99]
            tv_top1p = rolling_window_data[rolling_window_data['trade_volume'] >= percentile_99]['trade_volume'].sum()
            tv_top1p_rate = tv_top1p/tv
            trade_vol_top1p_ratio.append(tv_top1p_rate)

            # VARIABLE: volume power
            ask_sum = rolling_window_data[rolling_window_data['ask_bid'] == 'ASK']['trade_volume'].sum()
            bid_sum = rolling_window_data[rolling_window_data['ask_bid'] == 'BID']['trade_volume'].sum()
            vp = (bid_sum - ask_sum) / tv  
            # if ask_sum >= bid_sum:
            #     vp = -ask_sum/bid_sum
            # elif bid_sum > ask_sum:
            #     vp = bid_sum/ask_sum
            volume_power.append(vp)

            ask_sum_top1p = top1p[top1p['ask_bid'] == 'ASK']['trade_volume'].sum()
            bid_sum_top1p = top1p[top1p['ask_bid'] == 'BID']['trade_volume'].sum()
            vp_top1p = (bid_sum_top1p - ask_sum_top1p) / tv_top1p 
            volume_power_top1p.append(vp_top1p)    

            ep = rolling_window_data['trade_price'].iloc[-1]
            end_price.append(ep)

        # Create a new DataFrame to store the results
        d = pd.DataFrame({
            f'window_start_{interval}': window_start2,
            'window_end': window_end2,
            # f'realized_volatility_{interval}': realized_volatility,
            f'realized_volatility_mean0_{interval}': realized_volatility_mean0,
            f'num_trades_{interval}': num_trades,
            f'lowest_return_{interval}': lowest_return,
            f'highest_return_{interval}': highest_return,
            f'high_low_gap_{interval}': high_low_gap,
            f'trade_vol_{interval}': trade_vol,
            f'trade_vol_top1p_ratio_{interval}': trade_vol_top1p_ratio,
            f'volume_power_{interval}': volume_power,
            f'volume_power_top1p_{interval}': volume_power_top1p,
        })
        # print("###result_df:", result_df.head(3))
        # print("###result_df_{}:".format(interval), result_df_150.head(3))
        # result_df = pd.merge(result_df, result_df_150, on = 'window_start', how='inner')
        # print("###merged result_df:", result_df.head(3))
        result_df = pd.merge(result_df, d, on = 'window_end', how='inner')

    # agg = pd.merge(agg, d, on='time_id', how='left')  # 계산된 통계량을 기존 데이터프레임에 병합합니다.

    # 첫 1분 및 마지막 11분은 데이터가 complete 하지 않을 것이므로 삭제.
    start_time = result_df['time_id'].iloc[0]
    end_time = result_df['time_id'].iloc[-11]
    result_df_flt = result_df[(result_df['time_id'] > start_time) & (result_df['time_id'] < end_time)]

    return result_df_flt

### Feature Engineering (ORDERBOOK 부문)

In [4]:
def feature_engineering_orderbook(df_orderbook):

    # df_orderbook_pd2 = df_orderbook.compute() # dask dataframe => pandas dataframe
    df_orderbook_pd2 = df_orderbook.copy()

    gc.collect()
    # for testing:
    # df_orderbook_pd2 = df_orderbook_pd.iloc[:]

    df_orderbook_pd2['datetime'] = pd.to_datetime(df_orderbook_pd2['datetime'])
    # datetime을 1분 단위로 정리하는 변수 (dt_1m)을 만듬 (이는 데이터를 '정리'하는 데에 활용될 단위임)
    df_orderbook_pd2['dt_1m'] = df_orderbook_pd2['datetime'].dt.floor('1min').dt.strftime('%Y-%m-%d %H:%M:%S')
    df_orderbook_pd2['dt_1m'] = pd.to_datetime(df_orderbook_pd2['dt_1m'])
    df_orderbook_pd2.set_index('dt_1m', inplace=True, drop = False)

    # Define the size of the moving window (10 minutes in this case)
    window_size = pd.Timedelta(minutes=10)
    
    # Define the time range
    start_time = pd.to_datetime('00:00:00').time()
    end_time = pd.to_datetime('06:00:00').time()

    # Create an empty list to store features for each rolling window
    liq_last_1 = []
    liq_last_2 = []
    liq_last_5 = []
    # liq_last_10 = []
    # liq_last_15 = []
    ep_liq_1 =[]
    ep_liq_2 =[]
    ep_liq_5 =[]
    # ep_liq_10 =[]
    # ep_liq_15 =[]
    bidask_spread_0 = []
    bidask_spread_1 = []
    prices_30s_for_NN = []
    highest_possible_return = []
    window_start2 = []
    window_end2 = []
    temp_index = df_orderbook_pd2.index.unique().tolist()
    orderbook_bias_1 = []
    orderbook_bias_2 = []
    orderbook_bias_5 = []

    # Iterate through each time window using the rolling method

    print("df_orderbook_pd2.shape:", df_orderbook_pd2.shape)
    j = 0
    print("feature expansion from orderbook #1")
    for window_start in temp_index[:-10]:
        
        window_end = window_start + window_size

        if  (start_time <= window_start.time() <= end_time) or  (start_time <= window_end.time() <= end_time):
            continue 

        # Check the progress
        if j % 1000 ==0:
            print("main iteration:", j)

        window_start2.append(window_start)
        window_end2.append(window_end)
        # print("window_start:", window_start, " & window_end:", window_end)

        # Filter the data for each rolling window
        rolling_window_data = df_orderbook_pd2.loc[(df_orderbook_pd2.index >= window_start) & (df_orderbook_pd2.index < window_end)]
        # previous_price = rolling_window_data['ending_price_b1m'].iloc[0] # 10분 bin이 시작하기 직전 마지막 trade price 값을 의미.
        # rolling_window_data['log_return_from_pp'] = np.log(rolling_window_data['trade_price'] / previous_price)

        first_order_book = rolling_window_data.iloc[0]
        last_order_book = rolling_window_data.iloc[-1]

        best_buying_price_at_start = first_order_book['orderbook_ap_0']
        best_selling_price = rolling_window_data['orderbook_bp_0'].max()
        highest_possible_return2 = best_selling_price / best_buying_price_at_start
        highest_possible_return.append(highest_possible_return2) 

        # VARIABLE: liquidity measures
        wap_1 = (last_order_book['orderbook_bp_0'] * last_order_book['orderbook_as_0'] + last_order_book['orderbook_ap_0'] * last_order_book['orderbook_bs_0'])/(last_order_book['orderbook_bs_0'] + last_order_book['orderbook_as_0'])
        
        # liq_1 = 0
        # liq_2 = 0
        # liq_5 = 0
        # liq_10 = 0
        # liq_15 = 0

        # for i in range(1):
        #     liq_1 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

        # for i in range(2):
        #     liq_2 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

        # for i in range(5):
        #     liq_5 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

        # for i in range(10):
        #     liq_10 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

        data_liq = {}
        temp = 0
        for i in range(5):
            temp += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)
            if i+1 in [1, 2, 5]: # , 10, 15
                # print("liquidity calculatino checker_{}".format(i+1))
                variable_name = f'liq_{i+1}'
                data_liq[variable_name] = temp

        # print(data_liq)
        liq_last_1.append(data_liq['liq_1'])
        liq_last_2.append(data_liq['liq_2'])
        liq_last_5.append(data_liq['liq_5'])
        # liq_last_10.append(data_liq['liq_10'])
        # liq_last_15.append(data_liq['liq_15'])

        ep_wap_1 = (rolling_window_data['orderbook_bp_0'].mean() * rolling_window_data['orderbook_as_0'].mean() + rolling_window_data['orderbook_ap_0'].mean() * rolling_window_data['orderbook_bs_0'].mean())/(rolling_window_data['orderbook_bs_0'].mean() + rolling_window_data['orderbook_as_0'].mean())

        # entire_period_liq_1 = 0
        # entire_period_liq_2 = 0
        # entire_period_liq_5 = 0
        # entire_period_liq_10 = 0
        # entire_period_liq_15 = 0

        # for i in range(1):
        #     entire_period_liq_1 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

        # for i in range(2):
        #     entire_period_liq_2 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

        # for i in range(5):
        #     entire_period_liq_5 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

        # for i in range(10):
        #     entire_period_liq_10 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

        data_ep_liq = {}
        temp2 = 0
        for i in range(5):
            temp2 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)
            if i+1 in [1, 2, 5]: # , 10, 15
                variable_name = f'ep_liq_{i+1}'
                data_ep_liq[variable_name] = temp2

        ep_liq_1.append(data_ep_liq['ep_liq_1'])
        ep_liq_2.append(data_ep_liq['ep_liq_2'])
        ep_liq_5.append(data_ep_liq['ep_liq_5'])
        # ep_liq_10.append(data_ep_liq['ep_liq_10'])
        # ep_liq_15.append(data_ep_liq['ep_liq_15'])

        # VARIABLE: bidask spread
        ob_price_diff = []
        for i in range(14):
            ob_price_diff.append(last_order_book[f'orderbook_bp_{i}'] - last_order_book[f'orderbook_bp_{i+1}'])
            ob_price_diff.append(last_order_book[f'orderbook_ap_{i+1}'] - last_order_book[f'orderbook_ap_{i}'])
        tick_size = min(ob_price_diff)

        # 마지막 1분 정보만 추출해서 bid ask spread의 평균을 구하자 (마지막 타이밍의 orderbook만 쓰는 것보다 낫지 않을까?)
        last_1m = rolling_window_data.loc[(rolling_window_data.dt_1m == rolling_window_data['dt_1m'].iloc[-1])]

        ba_sp_0 = (last_1m['orderbook_ap_0'] - last_1m['orderbook_bp_0'])/tick_size
        bidask_spread_0.append(ba_sp_0.mean())
        ba_sp_1 = (last_1m['orderbook_ap_1'] - last_1m['orderbook_bp_1'])/tick_size
        bidask_spread_1.append(ba_sp_1.mean())
        
        ob_bias_1  = last_1m['orderbook_as_0'].sum() / last_1m['orderbook_bs_0'].sum() 
        ob_bias_2  = (last_1m['orderbook_as_0']+last_1m['orderbook_as_1']).sum() / (last_1m['orderbook_bs_0']+last_1m['orderbook_bs_1']).sum() 
        ob_bias_5  = (last_1m['orderbook_as_0']+last_1m['orderbook_as_1']+last_1m['orderbook_as_2']+last_1m['orderbook_as_3']+last_1m['orderbook_as_4']).sum()/ (last_1m['orderbook_bs_0']+last_1m['orderbook_bs_1']+last_1m['orderbook_bs_2']+last_1m['orderbook_bs_3']+last_1m['orderbook_bs_4']).sum()
        orderbook_bias_1.append(ob_bias_1)
        orderbook_bias_2.append(ob_bias_2)
        orderbook_bias_5.append(ob_bias_5)

        # liq_1 = [last_order_book['orderbook_bs_0']/(wap_1 - last_order_book['orderbook_bp_0']) + last_order_book['orderbook_as_0']/(last_order_book['orderbook_ap_0'] - wap_1)]
        # liq_1 = liq_1 +  

        # 30s prices 를 orderbook에서 가져와보자:
        returns_per_30s_temp = []
        distance = pd.Timedelta(seconds=30)
        first_order_book = rolling_window_data.iloc[0]
        first_wap = (first_order_book['orderbook_bp_0'] * first_order_book['orderbook_as_0'] + first_order_book['orderbook_ap_0'] * first_order_book['orderbook_bs_0'])/(first_order_book['orderbook_bs_0'] + first_order_book['orderbook_as_0'])
        for i in range(20):
            temp_30s_window_data = rolling_window_data.loc[(rolling_window_data.index >= window_start) & (rolling_window_data.index < window_start+distance*(i+1))]
            temp_mid_order_book = temp_30s_window_data.iloc[-1]
            mid_wap = (temp_mid_order_book['orderbook_bp_0'] * temp_mid_order_book['orderbook_as_0'] + temp_mid_order_book['orderbook_ap_0'] * temp_mid_order_book['orderbook_bs_0'])/(temp_mid_order_book['orderbook_bs_0'] + temp_mid_order_book['orderbook_as_0'])
            temp_return = np.log(mid_wap/first_wap)
            returns_per_30s_temp.append(temp_return)

        prices_30s_for_NN.append(returns_per_30s_temp)
        
        j= j+1
        gc.collect()

    # Create a new DataFrame to store the results
    result_df_orderbook = pd.DataFrame({
        'window_start': window_start2,
        'window_end': window_end2,
        'liq_last_1': liq_last_1,
        'liq_last_2': liq_last_2,
        'liq_last_5': liq_last_5,
        # 'liq_last_10': liq_last_10,
        # 'liq_last_15': liq_last_15,
        'ep_liq_1': ep_liq_1,
        'ep_liq_2': ep_liq_2,
        'ep_liq_5': ep_liq_5,
        # 'ep_liq_10': ep_liq_10,
        # 'ep_liq_15': ep_liq_15,
        'bidask_spread_0': bidask_spread_0,
        'bidask_spread_1': bidask_spread_1,
        'prices_30s_for_NN': prices_30s_for_NN,
        'highest_possible_return': highest_possible_return,
        'orderbook_bias_1': orderbook_bias_1,
        'orderbook_bias_2': orderbook_bias_2,
        'orderbook_bias_5': orderbook_bias_5,
    })

    # Reset the index of the result DataFrame
    result_df_orderbook.reset_index(drop=True, inplace=True)

    # time_id 만들어주기.
    result_df_orderbook['window_start'] = pd.to_datetime(result_df_orderbook['window_start'])
    result_df_orderbook['time_id'] = result_df_orderbook['window_start'].dt.floor('1min').dt.strftime('%Y-%m-%d %H:%M:%S')
    result_df_orderbook['time_id'] = pd.to_datetime(result_df_orderbook['time_id'])

    result_df_orderbook['dv6_highest_possible_return'] = result_df_orderbook['highest_possible_return'].shift(-10)

    print("feature expansion from orderbook #2 (entire time processing done)")


    ###### To-do : 450/300/150초 기준 변수 만들기.

    # Create an empty list to store features for each rolling window

    time_intervals = [60, 300]

    for interval in time_intervals:

        print("feature expansion from orderbook #3 ({}s beginning)".format(interval))

        liq_last_1 = []
        liq_last_2 = []
        liq_last_5 = []
        # liq_last_10 = []
        # liq_last_15 = []
        ep_liq_1 =[]
        ep_liq_2 =[]
        ep_liq_5 =[]
        # ep_liq_10 =[]
        # ep_liq_15 =[]        
        # bidask_spread_0 = []
        # bidask_spread_1 = []
        window_start2 = []
        window_end2 = []
        j=0
        for window_start in temp_index[:-10]:
            window_end = window_start + window_size

            if  (start_time <= window_start.time() <= end_time) or  (start_time <= window_end.time() <= end_time):
                continue             

            # Check the progress
            if j % 1000 ==0:
                print("subperiod_{} iteration:".format(interval), j)

            window_start2.append(window_start)
            window_end2.append(window_end)
            # Filter the data for each rolling window
            rolling_window_data = df_orderbook_pd2.loc[(df_orderbook_pd2['datetime'] >= window_end - pd.Timedelta(seconds=interval)) & (df_orderbook_pd2['datetime'] < window_end )]

            # VARIABLE: liquidity measures
            last_order_book = rolling_window_data.iloc[-1]
            wap_1 = (last_order_book['orderbook_bp_0'] * last_order_book['orderbook_as_0'] + last_order_book['orderbook_ap_0'] * last_order_book['orderbook_bs_0'])/(last_order_book['orderbook_bs_0'] + last_order_book['orderbook_as_0'])
            
            # liq_1 = 0
            # for i in range(1):
            #     liq_1 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

            # liq_2 = 0
            # for i in range(2):
            #     liq_2 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

            # liq_5 = 0
            # for i in range(5):
            #     liq_5 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

            # liq_10 = 0
            # for i in range(10):
            #     liq_10 += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)

            # liq_15 = 0
            data_liq = {}
            temp = 0
            for i in range(5):
                temp += last_order_book[f'orderbook_bs_{i}']/(wap_1 - last_order_book[f'orderbook_bp_{i}']) + last_order_book[f'orderbook_as_{i}']/(last_order_book[f'orderbook_ap_{i}'] - wap_1)
                if i+1 in [1, 2, 5]: #, 10, 15
                    variable_name = f'liq_{i+1}'
                    data_liq[variable_name] = temp


            liq_last_1.append(data_liq['liq_1'])
            liq_last_2.append(data_liq['liq_2'])
            liq_last_5.append(data_liq['liq_5'])
            # liq_last_10.append(data_liq['liq_10'])
            # liq_last_15.append(data_liq['liq_15'])

            ep_wap_1 = (rolling_window_data['orderbook_bp_0'].mean() * rolling_window_data['orderbook_as_0'].mean() + rolling_window_data['orderbook_ap_0'].mean() * rolling_window_data['orderbook_bs_0'].mean())/(rolling_window_data['orderbook_bs_0'].mean() + rolling_window_data['orderbook_as_0'].mean())

            # entire_period_liq_1 = 0
            # for i in range(1):
            #     entire_period_liq_1 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

            # entire_period_liq_2 = 0
            # for i in range(2):
            #     entire_period_liq_2 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

            # entire_period_liq_5 = 0
            # for i in range(5):
            #     entire_period_liq_5 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

            # entire_period_liq_10 = 0
            # for i in range(10):
            #     entire_period_liq_10 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)

            data_ep_liq = {}
            temp2 = 0
            # entire_period_liq_15 = 0
            for i in range(5):
                temp2 += rolling_window_data[f'orderbook_bs_{i}'].mean()/(ep_wap_1 - rolling_window_data[f'orderbook_bp_{i}'].mean()) + rolling_window_data[f'orderbook_as_{i}'].mean()/(rolling_window_data[f'orderbook_ap_{i}'].mean() - ep_wap_1)
                if i+1 in [1,2, 5]: # , 10, 15
                    variable_name = f'ep_liq_{i+1}'
                    data_ep_liq[variable_name]=temp2
            ep_liq_1.append(data_ep_liq['ep_liq_1'])
            ep_liq_2.append(data_ep_liq['ep_liq_2'])
            ep_liq_5.append(data_ep_liq['ep_liq_5'])
            # ep_liq_10.append(data_ep_liq['ep_liq_10'])
            # ep_liq_15.append(data_ep_liq['ep_liq_15'])

            # # VARIABLE: bidask spread
            # ob_price_diff = []
            # for i in range(14):
            #     ob_price_diff.append(last_order_book[f'orderbook_bp_{i}'] - last_order_book[f'orderbook_bp_{i+1}'])
            #     ob_price_diff.append(last_order_book[f'orderbook_ap_{i+1}'] - last_order_book[f'orderbook_ap_{i}'])
            # tick_size = min(ob_price_diff)

            # # 마지막 1분 정보만 추출해서 bid ask spread의 평균을 구하자 (마지막 타이밍의 orderbook만 쓰는 것보다 낫지 않을까?)
            # last_1m = rolling_window_data.loc[(rolling_window_data.dt_1m == rolling_window_data['dt_1m'].iloc[-1])]

            # ba_sp_0 = (last_1m['orderbook_ap_0'] - last_1m['orderbook_bp_0'])/tick_size
            # bidask_spread_0.append(ba_sp_0.mean())
            # ba_sp_1 = (last_1m['orderbook_ap_1'] - last_1m['orderbook_bp_1'])/tick_size
            # bidask_spread_1.append(ba_sp_1.mean())
            
            j = j+1
            gc.collect()

        # Create a new DataFrame to store the results
        d = pd.DataFrame({
        f'window_start_{interval}': window_start2,
        'window_end': window_end2,
        f'liq_last_1_{interval}': liq_last_1,
        f'liq_last_2_{interval}': liq_last_2,
        f'liq_last_5_{interval}': liq_last_5,
        # f'liq_last_10_{interval}': liq_last_10,
        # f'liq_last_15_{interval}': liq_last_15,
        f'ep_liq_1_{interval}': ep_liq_1,
        f'ep_liq_2_{interval}': ep_liq_2,
        f'ep_liq_5_{interval}': ep_liq_5,
        # f'ep_liq_10_{interval}': ep_liq_10,
        # f'ep_liq_15_{interval}': ep_liq_15,
        # f'bidask_spread_0_{interval}': bidask_spread_0,
        # f'bidask_spread_1_{interval}': bidask_spread_1,
        })

        result_df_orderbook = pd.merge(result_df_orderbook, d, on = 'window_end', how='inner')
        gc.collect()

    # 첫 1분 및 마지막 11분은 데이터가 complete 하지 않을 것이므로 삭제.
    start_time = result_df_orderbook['time_id'].iloc[0]
    end_time = result_df_orderbook['time_id'].iloc[-11]
    result_df_orderbook_flt = result_df_orderbook[(result_df_orderbook['time_id'] > start_time) & (result_df_orderbook['time_id'] < end_time)]

    return result_df_orderbook_flt

    # # Orderbook 10분 정리 dataframe을 중간 저장.
    # result_df_orderbook_flt.to_csv(working_directory + "output\\{}_sum_orderbook_10m.csv".format(coin), index=False)

    # ## result_df & result_df_orderbook 두개를 concat 하자.
    # result_df_orderbook_flt = result_df_orderbook_flt.drop(columns = ['window_end', 'time_id'])
    # combined_result_df = pd.merge(result_df_flt, result_df_orderbook_flt, on='window_start', suffixes=('_ticker', '_orderbook'))



In [5]:
import gc

vn = 8 ### 같은 데이터셋 안에서도 vn이 다를 수 있음.. (feature의 변화라던지..) 

df = df.compute() ############################################# dask dataframe 이면 pandas dataframe으로 바꾸어주어야.

# Convert float64 columns to float32
float_columns = df.select_dtypes(include=['float64'])
df = df.assign(**{col: df[col].astype('float32') for col in float_columns.columns})
int64_columns = df.select_dtypes(include=['int64'])
df = df.assign(**{col: df[col].astype('int32') for col in int64_columns.columns})

df_orderbook = df[df['type_websocket']=="orderbook"]
df_ticker = df[df['type_websocket']=="ticker"]

df_ticker = df_ticker.set_index('sys_datetime', drop=False)
df_orderbook = df_orderbook.set_index('sys_datetime', drop=False)

gc.collect()
# df_features_ticker = pd.read_csv(working_directory+"output\\BTC_sum_ticker_10m_202303271051_v7.csv")
df_features_ticker = feature_engineering_ticker(df_ticker)
# df_features_ticker['window_start'] = pd.to_datetime(df_features_ticker['window_start'])
df_features_ticker['window_end'] = pd.to_datetime(df_features_ticker['window_end'])
df_features_ticker.to_csv(working_directory + "output\\{}_sum_ticker_10m_{}_v{}.csv".format(coin, data_id, vn), index=False)

df_features_orderbook = feature_engineering_orderbook(df_orderbook)
df_features_orderbook.to_csv(working_directory + "output\\{}_sum_orderbook_10m_{}_v{}.csv".format(coin, data_id, vn), index=False)
# df_features_orderbook = pd.read_csv("output\\BTC_sum_orderbook_10m_202312122122_v7.csv")

# df_features_orderbook = df_features_orderbook.drop(columns = ['window_end', 'time_id'])
df_features_orderbook = df_features_orderbook.drop(columns = ['window_start', 'time_id'])

# df_features_orderbook['window_start'] = pd.to_datetime(df_features_orderbook['window_start'])
df_features_orderbook['window_end'] = pd.to_datetime(df_features_orderbook['window_end'])

combined_result_df = pd.merge(df_features_ticker, df_features_orderbook, on='window_end', suffixes=('_ticker', '_orderbook'))

## 추가변수 만들기
# combined_result_df['realized_volatility_30s'] = np.nan
# combined_result_df['dv4_realized_volatility_30s'] = np.nan
combined_result_df['prices_30s_for_NN_onlyprices'] = np.nan
combined_result_df['prices_30s_for_NN_onlyprices'] = combined_result_df['prices_30s_for_NN_onlyprices'].astype(str)

num_rows = combined_result_df.shape[0]
for i in range(num_rows):
    temp_series = combined_result_df['prices_30s_for_NN'].iloc[i]
    # # Split the string into a list based on the "\n" marker
    # tokenized_list = temp_series.split("\n")
    # # Remove any leading or trailing whitespace from each element
    # tokenized_list = [item.strip() for item in tokenized_list]
    # # Assuming you already have a list named "numbers"
    # if tokenized_list:
    #     tokenized_list.pop(0)  # This will remove the first item in the list
    #     tokenized_list.pop(-1)  # This will remove the first item in the list
    # # Extract the return numbers using list comprehension
    # return_numbers = [float(item.split()[-1]) for item in tokenized_list]
    # return_numbers = temp_series
    # temp_len = len(return_numbers)
    # squared_numbers = [x ** 2 for x in return_numbers]
    # sum_of_squared_numbers = sum(squared_numbers)
    # mean_of_sum_of_squared_numbers = sum_of_squared_numbers/temp_len
    # temp_vol = mean_of_sum_of_squared_numbers ** 0.5
    # combined_result_df.at[i, 'realized_volatility_30s'] = temp_vol
    return_numbers_str = ' '.join(str(num) for num in temp_series)
    combined_result_df.at[i, 'prices_30s_for_NN_onlyprices'] = return_numbers_str
    gc.collect()

# combined_result_df['dv4_realized_volatility_30s'] = combined_result_df['realized_volatility_30s'].shift(-10)

## Additional feature engineering (ex: TVLQ)
combined_result_df['trade.tau'] = np.sqrt(1 / combined_result_df['num_trades'])

combined_result_df['tvpl1'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_1']
combined_result_df['tvpl2'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_2'] 
combined_result_df['tvpl5'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_5'] 
# combined_result_df['tvpl10'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_10'] 
# combined_result_df['tvpl15'] = combined_result_df['trade_vol'] / combined_result_df['liq_last_15']

combined_result_df['tvpl_ep1'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_1']
combined_result_df['tvpl_ep2'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_2']
combined_result_df['tvpl_ep5'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_5']
# combined_result_df['tvpl_ep10'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_10']
# combined_result_df['tvpl_ep15'] = combined_result_df['trade_vol'] / combined_result_df['ep_liq_15']

# 아래 작업시 빠른 작업을 위해 10분씩 정리된 dataframe을 중간 저장.
combined_result_df.to_csv(working_directory + "output\\{}_sum_both_10m_{}_v{}.csv".format(coin, data_id, vn), index=False)

c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


###interval: 60


c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
No previous price.
###interval: 300
df_orderbook_pd2.shape: (10323127, 100)
feature expansion from orderbook #1
main iteration: 0


c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in float_scalars


main iteration: 1000
main iteration: 2000
main iteration: 3000
main iteration: 4000
main iteration: 5000
main iteration: 6000
main iteration: 7000
main iteration: 8000
main iteration: 9000
main iteration: 10000
main iteration: 11000
main iteration: 12000
main iteration: 13000
main iteration: 14000
main iteration: 15000
main iteration: 16000
main iteration: 17000
main iteration: 18000
main iteration: 19000
main iteration: 20000
main iteration: 21000
main iteration: 22000
main iteration: 23000
main iteration: 24000
main iteration: 25000
main iteration: 26000
main iteration: 27000
main iteration: 28000
main iteration: 29000
main iteration: 30000
main iteration: 31000
main iteration: 32000
main iteration: 33000
main iteration: 34000
main iteration: 35000
main iteration: 36000
main iteration: 37000
main iteration: 38000
main iteration: 39000
main iteration: 40000
main iteration: 41000
main iteration: 42000
main iteration: 43000
main iteration: 44000
main iteration: 45000
main iteration: 460

c:\Users\hhkim\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:291: RuntimeWarning: divide by zero encountered in float_scalars


subperiod_60 iteration: 1000
subperiod_60 iteration: 2000
subperiod_60 iteration: 3000
subperiod_60 iteration: 4000
subperiod_60 iteration: 5000
subperiod_60 iteration: 6000
subperiod_60 iteration: 7000
subperiod_60 iteration: 8000
subperiod_60 iteration: 9000
subperiod_60 iteration: 10000
subperiod_60 iteration: 11000
subperiod_60 iteration: 12000
subperiod_60 iteration: 13000
subperiod_60 iteration: 14000
subperiod_60 iteration: 15000
subperiod_60 iteration: 16000
subperiod_60 iteration: 17000
subperiod_60 iteration: 18000
subperiod_60 iteration: 19000
subperiod_60 iteration: 20000
subperiod_60 iteration: 21000
subperiod_60 iteration: 22000
subperiod_60 iteration: 23000
subperiod_60 iteration: 24000
subperiod_60 iteration: 25000
subperiod_60 iteration: 26000
subperiod_60 iteration: 27000
subperiod_60 iteration: 28000
subperiod_60 iteration: 29000
subperiod_60 iteration: 30000
subperiod_60 iteration: 31000
subperiod_60 iteration: 32000
subperiod_60 iteration: 33000
subperiod_60 iterat